In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

--2025-11-25 20:14:54--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-25T20%3A56%3A51Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-25T19%3A56%3A48Z&ske=2025-11-25T20%3A56%3A51Z&sks=b&skv=2018-11-09&sig=siRLrMoBwllP%2FP8vIavhoFWryZTlVHNzXH8KTiyjIQA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDEwMzQ5NCwibmJmIjoxNzY0MTAxNjk0LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlv

In [3]:
!unzip data.zip

Archive:  data.zip
replace data/test/curly/03312ac556a7d003f7570657f80392c34.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Conv2d: 3 input channels, 32 output channels, kernel size 3
        # Input shape: (3, 200, 200)
        # Output shape after conv: (32, 198, 198)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.relu = nn.ReLU()

        # MaxPool2d: kernel size 2
        # Output shape after pool: (32, 99, 99)
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Flatten
        # 32 * 99 * 99 = 313632
        self.flatten = nn.Flatten()

        # Linear layer: 32*99*99 -> 64
        self.fc1 = nn.Linear(32 * 99 * 99, 64)

        # Output layer: 64 -> 1
        self.fc2 = nn.Linear(64, 1)

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

def create_model_and_optimizer():
    model = CNN()
    optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
    criterion = nn.BCELoss()#nn.BCEWithLogitsLoss()
    return model, optimizer, criterion



```
nn.MSELoss(): This is typically used for regression problems, not binary classification.
nn.BCEWithLogitsLoss(): This is also a Binary Cross-Entropy Loss, but it expects raw, unscaled scores (logits) as input, and it internally applies the sigmoid function for numerical stability. If your SimpleCNN's forward method returned self.fc2(x) directly (without the torch.sigmoid), then nn.BCEWithLogitsLoss() would be the preferred and more numerically stable choice.
nn.CrossEntropyLoss(): This is primarily used for multi-class classification problems, not binary classification, and it also expects logits as input.
nn.CosineEmbeddingLoss(): This loss function is used for learning embeddings and measuring the similarity between two inputs, which is not applicable here.
So, while nn.BCEWithLogitsLoss() is conceptually close, given your current model's architecture which includes sigmoid in the forward pass, nn.BCELoss() is the appropriate loss function to use.
```



In [ ]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
model, optimizer, criterion = create_model_and_optimizer()
#print(model)
#print(optimizer)

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional Layer
        # Input channels: 3 (RGB image)
        # Output channels: 32
        # Kernel size: (3, 3)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        # Max Pooling Layer
        # Kernel size: (2, 2)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        # Calculate the size of the flattened features after conv and pool
        # Input H, W = 200, 200
        # After conv1: H = (200 - 3 + 0)/1 + 1 = 198, W = (200 - 3 + 0)/1 + 1 = 198
        # After pool: H = 198 / 2 = 99, W = 198 / 2 = 99
        # Flattened features: 32 (channels) * 99 (H) * 99 (W) = 313632
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        # Output layer for binary classification
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Apply convolution and ReLU activation
        x = F.relu(self.conv1(x))
        # Apply max pooling
        x = self.pool(x)

        # Flatten the feature map into a vector
        # x.size(0) gets the batch size, -1 infers the remaining dimension
        x = x.view(x.size(0), -1)

        # Apply first fully connected layer and ReLU activation
        x = F.relu(self.fc1(x))
        # Apply second fully connected (output) layer with Sigmoid for binary classification
        x = torch.sigmoid(self.fc2(x))
        return x


def create_model_and_optimizer_v2():
    model = CNN()
    optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
    criterion = nn.BCELoss()#nn.BCEWithLogitsLoss()
    return model, optimizer, criterion


In [38]:

from torchsummary import summary
model, optimizer, criterion = create_model_and_optimizer()

In [39]:
if torch.cuda.is_available():
    model.to('cuda')

summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
           Sigmoid-8                    [-1, 1]               0
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------
Total parameters: 20073473


In [40]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [41]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate model, optimizer, and criterion
model, optimizer, criterion = create_model_and_optimizer()
model.to(device)

# Load datasets
train_dataset = datasets.ImageFolder('data/train', transform=train_transforms)
validation_dataset = datasets.ImageFolder('data/test', transform=train_transforms)

# Create data loaders
batch_size = 20 # Changed batch size to 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [42]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (outputs > 0.5).float() # outputs already have sigmoid applied in SimpleCNN
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (outputs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6512, Acc: 0.6188, Val Loss: 0.6450, Val Acc: 0.6119
Epoch 2/10, Loss: 0.5314, Acc: 0.7238, Val Loss: 0.6370, Val Acc: 0.6617
Epoch 3/10, Loss: 0.5057, Acc: 0.7450, Val Loss: 0.5866, Val Acc: 0.6517
Epoch 4/10, Loss: 0.4690, Acc: 0.7662, Val Loss: 0.6053, Val Acc: 0.6517
Epoch 5/10, Loss: 0.3914, Acc: 0.8263, Val Loss: 0.5877, Val Acc: 0.6915
Epoch 6/10, Loss: 0.3291, Acc: 0.8525, Val Loss: 0.8346, Val Acc: 0.6816
Epoch 7/10, Loss: 0.3731, Acc: 0.8325, Val Loss: 0.6629, Val Acc: 0.6965
Epoch 8/10, Loss: 0.2460, Acc: 0.9025, Val Loss: 0.6110, Val Acc: 0.7363
Epoch 9/10, Loss: 0.2911, Acc: 0.8650, Val Loss: 0.5319, Val Acc: 0.7164
Epoch 10/10, Loss: 0.1603, Acc: 0.9450, Val Loss: 0.6369, Val Acc: 0.7562


In [46]:
import numpy as np

median_train_accuracy = np.median(history['acc'])
print(f"3 Median of training accuracy for all epochs: {median_train_accuracy:.4f}")

3 Median of training accuracy for all epochs: 0.8294


In [47]:
import numpy as np

std_dev_train_loss = np.std(history['loss'])
print(f"4 Standard deviation of training loss for all epochs: {std_dev_train_loss:.4f}")

4 Standard deviation of training loss for all epochs: 0.1396


In [50]:
import numpy as np

mean_val_loss = np.mean(history['val_loss'])
print(f"5 Mean of validation loss for all epochs: {mean_val_loss:.4f}")

5 Mean of validation loss for all epochs: 0.6339


In [49]:
import numpy as np

# Get the validation accuracy for the last 5 epochs
last_5_val_acc = history['val_acc'][-5:]

# Calculate the average of these accuracies
average_val_acc_last_5_epochs = np.mean(last_5_val_acc)

print(f"6 Average of validation accuracy for the last 5 epochs: {average_val_acc_last_5_epochs:.4f}")

6 Average of validation accuracy for the last 5 epochs: 0.7174
